In [ ]:
# Inception-v3 Code
# Importing Libraries
import keras 
from keras.preprocessing import image
from keras.applications import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
import cv2
import gc
import numpy as np
import matplotlib.pyplot as plt
import keras.layers as layers

# Dimensions of our images.
img_width, img_height = 256,256


# Input and training parameters
train_data_dir = '/kaggle/input/hne/hfolds/f1/train/'
validation_data_dir = '/kaggle/input/hne/hfolds/f1/validation/'
nb_train_samples = 2274
nb_validation_samples = 750
epochs = 50
batch_size = 12
chanDim = -1


if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


In [ ]:
# Use Keras Inception-V3 backbone
def build_model(backbone, lr=1e-4):
    model = Sequential()
    model.add(backbone)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.5))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(1, activation='sigmoid'))
    
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(lr=lr),
        metrics=['accuracy']
    )
    
    return model

In [ ]:
K.clear_session()
gc.collect()

# Not initializing weights
inceptionv3 = InceptionV3(
    weights=None,
    include_top=False,
    input_shape=(256,256,3)
)

model = build_model(inceptionv3 ,lr = 1e-4)
model.summary()

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

validation_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

history=model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json

with open('InceptionV3.json', 'w') as f:
    json.dump(str(history.history), f)
    
# Plotting Accuracy Curve
history_df = pd.DataFrame(history.history)
history_df[['accuracy', 'val_accuracy']].plot()
plt.title('Training accuracy vs Validation accuracy of Inception-V3 for HnE')
plt.ylabel('Accuracy')
plt.xlabel('No of epochs')

In [ ]:
# Plotting Loss Curve 
history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
plt.title('Training loss vs Validation loss of Inception-V3 for HnE')
plt.ylabel('Loss')
plt.xlabel('No of epochs')

In [ ]:
# Testing
test_data_dir = '/kaggle/input/hne/hfolds/f1/test'
test_data_generator = ImageDataGenerator(rescale=1./255)
test_generator = test_data_generator.flow_from_directory(
    test_data_dir,
    target_size=(256, 256),
    batch_size=1,
    class_mode="binary", 
    shuffle=False)

In [ ]:
# Saving Model weights
MODEL_FILE = "InceptionV3.h5"
model.save_weights(MODEL_FILE)

# Getting predictions for test images
probabilities = model.predict_generator(test_generator, 762)
print(probabilities)

In [ ]:
# Thresholding prediction probabilities at 0.5
prob=[]
print(len(probabilities))
for i in range(0, 762):
    if (probabilities[i][0] > 0.5):
        prob.append(1)
    else:
        prob.append(0)
    
print(prob)

In [ ]:
# Generating true labels
labels=[]
# Benign
for i in range(0, 516):
    labels.append(0)
    
# Malignant
for i in range(0, 246):
    labels.append(1)

In [ ]:
from keras.utils import np_utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np

# Print the confusion Matrix
cm = confusion_matrix(labels, prob)
print(cm)